## Load & Agg

In [1]:
# tvl_growth_dashboard.py

"""
Revised TVL Growth Dashboard
----------------------------
1) Sorts the legend according to each protocol's rank (best to worst).
2) Removes extra labeling from the trace name (no last-week rank in the name).
3) Uses a static color for each (category, protocol) pair.
4) Includes an "All" category that shows all protocols, works with every measure,
   and supports 'aggregate_growth_score.'
"""

import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import plotly.express as px

# File paths
FINAL_TVL_PATH = "final_tvl_aggregate.csv"
GROWTH_METRICS_PATH = "protocol_growth_metrics.csv"

# Load data
df_tvl_agg = pd.read_csv(FINAL_TVL_PATH)
df_growth = pd.read_csv(GROWTH_METRICS_PATH)

print(f"Loaded df_tvl_agg => {df_tvl_agg.shape}")
print(f"Loaded df_growth => {df_growth.shape}")

# Our core measures + an aggregate
BASE_GROWTH_COLS = [
    "tvl_percent_growth",
    "avg_tvl_rank",
    "logistic_growth_rate",
    "avg_mom_growth_abs",
    "avg_mom_growth_percent"
]
ALL_GROWTH_COLS = BASE_GROWTH_COLS + ["aggregate_growth_score"]

def generate_rank_columns(growth_df):
    """
    1) Rank each measure (ascending for 'avg_tvl_rank', descending for the rest).
    2) Create 'aggregate_growth_score' = average of those ranks (lower is better).
    3) Create 'aggregate_growth_score_rank' = rank of that aggregate score.
    """
    df_ranked = growth_df.copy()

    # 1) Rank each measure
    for col in BASE_GROWTH_COLS:
        if col in df_ranked.columns:
            ascending_order = (col == "avg_tvl_rank")
            df_ranked[f"{col}_rank"] = df_ranked[col].rank(
                method="min", ascending=ascending_order
            ).astype(int)
        else:
            # If missing, fill as NaN
            df_ranked[f"{col}_rank"] = float("nan")

    # 2) The aggregate growth score is the simple average of these rank columns
    rank_cols = [f"{c}_rank" for c in BASE_GROWTH_COLS]
    df_ranked["aggregate_growth_score"] = df_ranked[rank_cols].mean(axis=1)

    # 3) Rank that aggregate (ascending => lower is better)
    df_ranked["aggregate_growth_score_rank"] = df_ranked["aggregate_growth_score"].rank(
        method="min", ascending=True
    ).astype(int)

    return df_ranked

# Build a combined category list with an "All" option
all_categories = sorted(df_growth["protocol_category"].unique().tolist())
all_categories.insert(0, "All")

# ------- Create static (category, protocol) color mapping -------
df_pairs = df_growth[["protocol_category", "protocol_name"]].drop_duplicates()

# Also consider "All" for every protocol
all_protos = df_growth["protocol_name"].unique()
rows_for_all = [
    {"protocol_category": "All", "protocol_name": p} for p in all_protos
]
df_pairs = pd.concat([df_pairs, pd.DataFrame(rows_for_all)], ignore_index=True)

color_palette = px.colors.qualitative.Plotly  # Using Plotly's palette
pair_list = df_pairs.to_records(index=False)
color_map = {}
for i, (cat, proto) in enumerate(pair_list):
    color_map[(cat, proto)] = color_palette[i % len(color_palette)]


# Utility to figure out the "rank column" for a measure
def get_rank_col(measure):
    """
    If measure is in BASE_GROWTH_COLS, rank col is measure + '_rank'.
    If measure is 'aggregate_growth_score', rank col = 'aggregate_growth_score_rank'.
    If measure already ends with '_rank', we assume it's itself.
    """
    if measure in BASE_GROWTH_COLS:
        return f"{measure}_rank"
    elif measure == "aggregate_growth_score":
        return "aggregate_growth_score_rank"
    else:
        # measure might already be a rank col name
        return measure

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Protocol TVL Growth Dashboard"),
    html.Div([
        html.Label("Select Category:"),
        dcc.Dropdown(
            id='category-dropdown',
            options=[{"label": c, "value": c} for c in all_categories],
            value=("All" if len(all_categories) > 0 else None)
        )
    ]),
    html.Div([
        html.Label("Select Growth Measure:"),
        dcc.Dropdown(
            id='growth-measure-dropdown',
            options=[{"label": c, "value": c} for c in ALL_GROWTH_COLS],
            value='tvl_percent_growth'
        )
    ]),
    html.Div([
        html.Label("Top N Protocols:"),
        dcc.Input(
            id="top-n-input",
            type="number",
            value=10,
            min=1,
            max=50,
            step=1
        )
    ]),
    html.Button("Download CSV", id="download-button", n_clicks=0),
    dcc.Download(id="download-dataframe-csv"),
    dcc.Graph(id="tvl-chart"),
])

@app.callback(
    Output("tvl-chart", "figure"),
    [
        Input("category-dropdown", "value"),
        Input("growth-measure-dropdown", "value"),
        Input("top-n-input", "value")
    ]
)
def render_tvl_chart(selected_cat, selected_measure, top_n):
    if not selected_cat or not selected_measure or not top_n or top_n <= 0:
        return go.Figure()

    # If "All" => use all df_growth, else slice by category
    if selected_cat == "All":
        cat_growth_df = df_growth.copy()
    else:
        cat_growth_df = df_growth[df_growth["protocol_category"] == selected_cat]

    if cat_growth_df.empty:
        return go.Figure()

    # Compute ranks
    df_with_ranks = generate_rank_columns(cat_growth_df)

    # Ascending if measure is 'avg_tvl_rank', endswith("_rank"), or == 'aggregate_growth_score'
    if (selected_measure == "avg_tvl_rank"
        or selected_measure.endswith("_rank")
        or selected_measure == "aggregate_growth_score"):
        best_protocols = df_with_ranks.nsmallest(top_n, selected_measure)["protocol_name"].tolist()
    else:
        best_protocols = df_with_ranks.nlargest(top_n, selected_measure)["protocol_name"].tolist()

    # Now reorder these protocols for the legend by the measure's rank col
    measure_rank_col = get_rank_col(selected_measure)
    best_subset = df_with_ranks[df_with_ranks["protocol_name"].isin(best_protocols)].copy()
    best_subset.sort_values(by=measure_rank_col, ascending=True, inplace=True)
    # Extract the protocol names in rank order
    final_protocol_order = best_subset["protocol_name"].tolist()

    chart_data = []
    for proto in final_protocol_order:
        # Filter down to just this protocol in df_tvl_agg
        df_proto = df_tvl_agg[df_tvl_agg["protocol_name"] == proto].copy()
        df_proto["dt"] = pd.to_datetime(df_proto["dt"], errors="coerce")
        df_proto.set_index("dt", inplace=True)
        df_proto.sort_index(inplace=True)
        tvl_series = df_proto["app_token_tvl_usd"].resample("D").last().ffill()

        # Color by (selected_cat, proto) or (All, proto)
        color_key = (selected_cat, proto)
        line_color = color_map.get(color_key, "#000000")

        chart_data.append(go.Scatter(
            x=tvl_series.index,
            y=tvl_series.values,
            mode="lines",
            name=proto,  # No extra label text, just the protocol name
            line=dict(color=line_color)
        ))

    layout = go.Layout(
        title=f"{selected_cat}: Top {top_n} by {selected_measure}",
        xaxis=dict(title="Date"),
        yaxis=dict(title="TVL (USD)"),
        hovermode="closest"
    )
    return go.Figure(data=chart_data, layout=layout)

@app.callback(
    Output("download-dataframe-csv", "data"),
    [Input("download-button", "n_clicks")],
    [
        State("category-dropdown", "value"),
        State("growth-measure-dropdown", "value"),
        State("top-n-input", "value")
    ]
)
def download_selected_data(n_clicks, cat_value, measure_value, top_n_value):
    if n_clicks == 0 or not cat_value or not measure_value or top_n_value is None or top_n_value <= 0:
        return dash.no_update

    # "All" => entire df_growth, else slice
    if cat_value == "All":
        cat_growth_df = df_growth.copy()
    else:
        cat_growth_df = df_growth[df_growth["protocol_category"] == cat_value]

    df_with_ranks = generate_rank_columns(cat_growth_df)

    # Ascending vs. Descending
    if (measure_value == "avg_tvl_rank"
        or measure_value.endswith("_rank")
        or measure_value == "aggregate_growth_score"):
        best_protocols = df_with_ranks.nsmallest(top_n_value, measure_value)["protocol_name"].tolist()
    else:
        best_protocols = df_with_ranks.nlargest(top_n_value, measure_value)["protocol_name"].tolist()

    df_filtered = df_tvl_agg[df_tvl_agg["protocol_name"].isin(best_protocols)].copy()
    csv_data = df_filtered.to_csv(index=False)
    filename = f"{cat_value}_by_{measure_value}.csv"
    return dict(content=csv_data, filename=filename)

if __name__ == "__main__":
    app.run_server(debug=True, port=8050)

Loaded df_tvl_agg => (129791, 6)
Loaded df_growth => (410, 8)
